Lecture: AI I - Advanced 

Previous:
[**Chapter 2.1: Regularization**](../02_training/01_regularization.ipynb)

---

# Chapter 2.2: Hyperparameter Tuning

In the previous section, you learned various regularization techniques to prevent overfitting—dropout, batch normalization, early stopping, and weight decay. But each technique introduced new hyperparameters: What dropout rate should you use? How much weight decay? You probably chose values like 0.3 or 0.01 somewhat arbitrarily, perhaps based on common defaults. Now comes the question: How do you find the optimal configuration among thousands of possible combinations?

This is where **hyperparameter tuning** becomes essential. Instead of guessing or manually trying different values, we'll use systematic search algorithms to explore the hyperparameter space efficiently and find configurations that maximize our model's performance.

## What Are Hyperparameters?

**Hyperparameters** are configuration settings that you choose before training begins—they control the learning process itself, unlike model parameters (weights and biases) which are learned during training. Choosing good hyperparameters can mean the difference between a model that achieves 70% accuracy and one that reaches 95%.

## Hyperparameters can include:
### 1. Optimizer and Learning Rate

The **optimizer** determines how weights are updated (SGD, Adam, RMSprop), while the **learning rate** controls the step size. This is often the most critical hyperparameter—too high and training diverges, too low and learning is painfully slow. Learning rates typically range from $1e-5$ to $1e-1$ and are best searched on a logarithmic scale (since 0.001 to 0.01 is as significant a change as 0.01 to 0.1).

**Search strategy**: Log-uniform distribution <br>
**Typical range**: 1e-5 to 1e-1

### 2. Loss Function

For classification, you'll typically use CrossEntropyLoss, while regression uses MSELoss or L1Loss. The choice depends on your task rather than being a tunable parameter, though in some cases (e.g., imbalanced classification) you might compare weighted vs. unweighted loss functions.

**Search strategy**: Usually fixed by task, occasionally categorical choice

### 3. Epochs

The number of training iterations through the entire dataset. Too few epochs and your model underfits; too many and you waste time or overfit (though early stopping mitigates this). Modern practice often uses a high max_epochs with early stopping rather than tuning this directly.

**Search strategy**: Usually fixed with early stopping, occasionally discrete <br>  
**Typical range**: 50-500

### 4. Batch Size

How many samples are processed before updating weights. Smaller batches (16-32) add noise that can help generalization but slow training; larger batches (128-256) are faster but may converge to sharper minima. Batch size also interacts with learning rate—larger batches often need higher learning rates.

**Search strategy**: Categorical choice (powers of 2) <br>
**Typical values**: [16, 32, 64, 128]

### 6. Per-Layer Configuration

Each layer can have different settings:
- **Activation function**: ReLU is standard, but Leaky ReLU, ELU, or GELU might work better for specific tasks
- **Neuron count (hidden dimensions)**: Typically powers of 2 (32, 64, 128, 256, 512) for computational efficiency
- **Regularization parameters** (dropout rate, initialization) can also vary per layer

**Search strategy**: Categorical (activation) or discrete/categorical (neuron count)

### 7. Architecture

**Depth** (number of layers) and **shape** (how dimensions change between layers—pyramid, constant, hourglass):
- **Shallow networks (1-2 hidden layers)**: Simpler, less prone to overfitting, limited capacity
- **Deep networks (3+ hidden layers)**: More expressive, can learn hierarchical features, harder to train

**Search strategy**: Discrete (number of layers) + continuous/discrete (dimension pattern)

## Search Strategy

**Random search** tries random combinations—simple but inefficient. Grid search is systematic but exponentially expensive (3 values for 5 hyperparameters = 3^5 = 243 trials!).

**Bayesian optimization** is smarter: it builds a probabilistic model of how hyperparameters affect performance, then uses this model to suggest promising configurations. After each trial, it updates its belief about which regions of the hyperparameter space are worth exploring. This means it finds good configurations in far fewer trials than random or grid search.

**Optuna** implements several Bayesian optimization algorithms, with **Tree-structured Parzen Estimator (TPE)** as the default. Think of it as learning from experience: "High learning rates with small batch sizes performed poorly, so let's try lower learning rates next."

## Hyperparameter Tuning with Optuna

In [ ]:
import torch

from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from torch.utils.data import TensorDataset, DataLoader

torch.manual_seed(42)  # set random seed for reproducibility

iris = datasets.load_iris()
x = iris.data  # 4 features: sepal length, sepal width, petal length, petal width
y = iris.target  # 3 classes: setosa, versicolor, virginica

x_scaled = StandardScaler().fit_transform(x)
x_train, x_temp, y_train, y_temp = train_test_split(
    x_scaled, y, test_size=0.4, random_state=42, stratify=y
)
x_val, x_test, y_val, y_test = train_test_split(
    x_temp, y_temp, test_size=0.5, random_state=42, stratify=y_temp
)

batch_size = 16
train_loader = DataLoader(TensorDataset(torch.FloatTensor(x_train), torch.LongTensor(y_train)), batch_size=batch_size, shuffle=True)
val_loader = DataLoader(TensorDataset(torch.FloatTensor(x_val), torch.LongTensor(y_val)), batch_size=batch_size)
test_loader = DataLoader(TensorDataset(torch.FloatTensor(x_test), torch.LongTensor(y_test)), batch_size=batch_size)

### Flexible Model Architecture 

In [ ]:
import torch.nn as nn

class FlexibleClassifier(nn.Module):
    """
    Flexible neural network that can be configured with different architectures.
    Supports variable depth and layer dimensions.
    """
    def __init__(self, input_dim, hidden_dims, output_dim, dropout_rate):
        """
        Args:
            input_dim: Number of input features
            hidden_dims: List of hidden layer dimensions, e.g., [64, 32]
            output_dim: Number of output classes
            dropout_rate: Dropout probability
        """
        super().__init__()
        
        # Build layers dynamically
        layers = []
        prev_dim = input_dim
        
        for hidden_dim in hidden_dims:
            layers.append(nn.Linear(prev_dim, hidden_dim))
            layers.append(nn.ReLU())
            layers.append(nn.Dropout(dropout_rate))
            prev_dim = hidden_dim
        
        # Output layer
        layers.append(nn.Linear(prev_dim, output_dim))
        
        self.network = nn.Sequential(*layers)
    
    def forward(self, x):
        return self.network(x)

---

Lecture: AI I - Basics 

Exercise: [**Exercise 2.2: Hyperparameter Tuning**](../02_training/exercises/02_optimization.ipynb)

Next: [**Chapter 2.3: Ensemble Learning**](../02_training/03_ensemble.ipynb)